In [ ]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd
import json

db = database('trfc.db')

# Create dataclass definitions
@dataclass
class GameStats:
    game_no: int
    goals: int
    minutes: Optional[int]
    yellow_cards: int
    red_cards: int

@dataclass
class PlayerStats:
    name: str
    games: list[GameStats]

# Function to get player stats
def get_player_stats(db_conn, season: str) -> list[PlayerStats]:
    query = """
    WITH game_stats AS (
        SELECT 
            p.surname || ', ' || p.forename as player,
            r.game_no,
            COUNT(g.player_id) as goals,
            COUNT(yc.player_id) as yellows,
            COUNT(rc.player_id) as reds,
            COALESCE(rc.min_so, subs_off.sub_min_off, cgd.game_length, 90) - 
                COALESCE(subs_on.sub_min_on, 0) as minutes
        FROM results r
        LEFT JOIN player_apps pa
            ON r.game_date = pa.game_date
        LEFT JOIN players p
            ON pa.player_id = p.player_id
        LEFT JOIN goals g 
            ON r.game_date = g.game_date 
            AND pa.player_id = g.player_id
        LEFT JOIN cards_yellow yc 
            ON r.game_date = yc.game_date 
            AND pa.player_id = yc.player_id
        LEFT JOIN cards_red rc 
            ON r.game_date = rc.game_date 
            AND pa.player_id = rc.player_id
        LEFT JOIN subs_on 
            ON r.game_date = subs_on.game_date 
            AND pa.player_id = subs_on.player_on_id
        LEFT JOIN subs_off 
            ON r.game_date = subs_off.game_date 
            AND pa.player_id = subs_off.player_off_id
        LEFT JOIN cup_game_details cgd 
            ON r.game_date = cgd.game_date
        WHERE r.season = ?
        GROUP BY player, r.game_no
    )
    SELECT 
        player,
        json_group_array(
            json_object(
                'game_no', game_no,
                'goals', goals,
                'minutes', minutes,
                'yellow_cards', yellows,
                'red_cards', reds
            )
        ) as games
    FROM game_stats
    GROUP BY player
    ORDER BY player
    """
    
    cursor = db_conn.execute(query, [season])
    results = cursor.fetchall()
    
    return [
        PlayerStats(
            name=row[0],
            games=[GameStats(**g) for g in json.loads(row[1])]
        )
        for row in results
    ]

get_player_stats(db, '2024/25')

In [27]:
def get_minutes_class(minutes):
    """Return CSS class based on minutes played"""
    if minutes is None or minutes == 0:
        return "bg-gray-200"  # DNP
    if minutes < 15:
        return "bg-red-200"   # Brief appearance
    if minutes < 45:
        return "bg-yellow-200"  # Less than half
    if minutes < 90:
        return "bg-green-200"  # Partial game
    return "bg-blue-200"      # Full game

def format_goals(goals):
    """Format goals for display"""
    return str(goals) if goals > 0 else ""

In [28]:
stats = get_player_stats(db, '2024/25')

max_game = max(
    game.game_no
    for player in stats
    for game in player.games
)


header_row = Tr(
    Th("Player", cls="text-left p-2"),*[Th(str(i), cls="p-2 text-center") for i in range(1, max_game + 1)]
)

player_rows = []
for player in stats:
    # Create a dict of game_no -> game for easier lookup
    games_dict = {game.game_no: game for game in player.games}
    
    # Create cells for each game
    game_cells = []
    for game_no in range(1, max_game + 1):
        game = games_dict.get(game_no)
        if game:
            cell_class = get_minutes_class(game.minutes)
            cell_content = format_goals(game.goals)
            title = f"Minutes: {game.minutes}"
        else:
            cell_class = "bg-white"
            cell_content = ""
            title = "Not in squad"
        
        game_cells.append(
            Td(cell_content, 
                cls=f"p-2 text-center {cell_class}",
                title=title)
        )
    
    # Create row for this player
    player_rows.append(
        Tr(
            Td(player.name, cls="p-2 font-medium"),
            *game_cells,
            cls="border-b hover:bg-gray-50"
        )
    )

In [ ]:
Table(
    Thead(header_row),
    Tbody(*player_rows),
    cls="min-w-full border-collapse"
)